In [46]:
import pdfplumber
import os
import pandas as pd
from datetime import datetime
import json

with open('TABLA_CODIGO_NOMBRE_FARMACOS.json', 'r') as f:
    DICCIONARIO_CODIGO_NOMBRE_FARMACOS = json.load(f)

with open('COLUMNAS_FARMACOS_EN_EXCEL.json', 'r') as f:
    COLUMNAS_FARMACOS_EN_EXCEL = json.load(f) 

In [60]:
class Formateador:
    def __init__(self):
        pass

    def hacer_tabla_global(self):
        self.antibiogramas = self.obtener_datos_todos_los_pacientes()
        df = pd.DataFrame(self.antibiogramas, columns = ['Ingreso', 'Tipo muestra', 'Nº de Cultivo', 'Nº de Orden', 'Rut', 'Nombre', 'Servicio', 'Fecha Firma', 'Resultados'])
                                # 'Amikacina': [],
                                # 'Ampicilina': [],
                                # 'Aztreonam': [],
                                # 'Cloranfenicol': [],
                                # 'Ceftazidima': [],
                                # 'Cefazolina': [],
                                # 'NaN': [],
                                # 'Cefepima': [],
                                # 'Cefotaxima': [],
                                # 'Cefoxitina': [],
                                # 'Ciprofloxacino': [],
                                # 'Clindamicina': [],
                                # 'Colistina': [],
                                # 'Cotrimoxazol': [],
                                # 'Eritromicina': [],
                                # 'Ertapenem': [],
                                # 'Gentamicina': [],
                                # 'Gentamicina 120': [],
                                # 'Imipenem': [],
                                # 'Linezolid': [],
                                # 'Meropenem': [],
                                # 'Nitrofurantoina': [],
                                # 'Penicilina': [],
                                # 'Rifampicina': [],
                                # 'Ampicilina/Sulbactam': [],
                                # 'Sulperazona': [],
                                # 'Tazobactam/Piperacilina': [],
                                # 'Teicoplanina': [],
                                # 'Tigeciclina': [],
                                # 'Tobramicina': [],
                                # 'Levofloxacino': [],
                                # 'Minociclina': [],
                                # 'Daptomicina': [],
                                # 'Vancomicina': []}
        return df

    def formatear_como_df(self, tupla_datos_paciente):
        nombre_paciente, n_orden, rut, fecha_ingreso, fecha_firma, seccion, tipo_muestra, n_cultivo = tupla_datos_paciente[0]
        tablas_sensibilidades = tupla_datos_paciente[1]


        
        pass

    def obtener_datos_todos_los_pacientes(self):
        pacientes = []
        for nombre_archivo in os.listdir():
            if '.pdf' in nombre_archivo:
                paciente = self.obtener_datos_totales_un_paciente(nombre_archivo)
                pacientes.append(paciente)
        

        antibiogramas = [paciente for paciente in pacientes if type(paciente[-1]) == pd.DataFrame]
        return antibiogramas


    def obtener_datos_totales_un_paciente(self, nombre_archivo_paciente):
        datos_personales = self.obtener_datos_personales_paciente(nombre_archivo_paciente)
        resultados_sensibilidad = self.obtener_tabla_antibiograma(nombre_archivo_paciente)

        return [*datos_personales, resultados_sensibilidad]

    # Esta función obteine los datos personales de un paciente, retorna una tupla.
    def obtener_datos_personales_paciente(self, nombre_archivo_paciente):
        with pdfplumber.open(nombre_archivo_paciente) as pdf:
            datos_personales_relevantes = pdf.pages[0].extract_text().split('\n')[3:12]

            nombre_paciente = datos_personales_relevantes[0].split(':')[1][:-10]
            n_orden = datos_personales_relevantes[0].split(':')[-1]
            rut = datos_personales_relevantes[1].split(':')[-1]

            linea_ingreso = datos_personales_relevantes[4].split(' ')
            try:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d/%m/%Y %H:%M:%S')


            linea_firma = datos_personales_relevantes[5].split(' ')
            try:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d/%m/%Y %H:%M:%S')

            seccion = datos_personales_relevantes[6].split(':')[-1]
            tipo_muestra = datos_personales_relevantes[7].split(':')[-1]
            n_cultivo = datos_personales_relevantes[8].split(':', 1)[-1]

            return fecha_ingreso, tipo_muestra, n_cultivo, n_orden, rut, nombre_paciente, seccion, fecha_firma

    # Esta función itera en todo un pdf, para buscar las tablas que sean de antibiograma.
    def obtener_tabla_antibiograma(self, nombre_archivo_paciente):
        with pdfplumber.open(nombre_archivo_paciente) as pdf:
            for pagina in pdf.pages:
                tablas_totales_de_pagina = pagina.extract_table()
                if tablas_totales_de_pagina:
                    for tabla in tablas_totales_de_pagina:
                        tabla = tabla[0].split('\n')
                        if 'ANTIBIOGRAMA' in tabla:
                            df_formateado = self.formatear_tabla_antibiograma(tabla)
                            return df_formateado

                else:
                    return None 

    # Esta función construye la tabla de antibiograma para hacerla un Df. Retorna un Df
    def formatear_tabla_antibiograma(self, lista_tabla):
        separados = list(map(lambda x: x.split(' '), lista_tabla))
        nombre_tabla, headers = separados[0], separados[1]
        numero_cepas = [dato for dato in headers if dato.isnumeric()]

        columnas_headers = ['ANTIBIOTICOS']
        for numero_cepa in numero_cepas:
            columnas_headers.append(f'Cepa {numero_cepa}')
            columnas_headers.append(f'CIM')
        

        datos_susceptibilidad = separados[2:]
        for dato in datos_susceptibilidad:
            espacios_a_llenar_en_linea = len(columnas_headers) - len(dato)
            dato += [None] * espacios_a_llenar_en_linea

        df = pd.DataFrame(datos_susceptibilidad, columns = columnas_headers)
        df.index = [DICCIONARIO_CODIGO_NOMBRE_FARMACOS[farmaco] for farmaco in df.ANTIBIOTICOS]
        df.drop(columns = ['ANTIBIOTICOS', 'CIM'], inplace = True)
        return df
    


In [61]:
formateador = Formateador()
df = formateador.obtener_tabla_antibiograma('943891_OLGA_DEL_CARMEN_VERDUGO_CASTILLO.pdf')

In [62]:
df

,Cepa 1
CAF,Sensible
CFTXIMA,Sensible
IMI,Sensible
MERO,Sensible
SAM,Sensible
COTRI,Sensible
